In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd
import math
import random
import functools
import csv
import os

## ILP for the Unbounded Multiple Knapsack Problem with Processing Times and Multiple Starting Times

In [3]:
n = 5 # number of campaign
m = 100 # number of customers
d = 45 # days to be scheduled

s = [] # starting time
for i in range(n):
    s.append(random.sample(range(d), random.randint(1, 5)))
p = np.random.choice([5, 8, 15, 23, 30, 43, 56, 64], n) # processing time
c = np.random.randint(low = 10, high = 40, size = n) # cost
b = np.random.randint(low = 40, high = 80, size = n) # benefit
C = np.random.randint(low = 50, high = 100, size = n) # maximum cost
    
zippedList =  list(zip(s, p, c, b, C))
campaigns = pd.DataFrame(zippedList, columns = ['starting times', 'processing time' , 'cost', 'benefit', 'maximum cost'])
campaigns.index.names = ['name']
# print("Dataframe : " , campaigns, sep='\n')

# matrix with take-rates
T = np.random.random_sample((m, n))

In [8]:
# create empty model
model = gp.Model()

# add decision variables
x = model.addMVar((m,n), vtype=GRB.BINARY, name='x') # variable x_ij
z = model.addMVar((m,n,d), vtype=GRB.BINARY, name = 'z') # variable z_ijl
v = model.addMVar((n,d), vtype=GRB.BINARY, name = 'v') # variable v_jl
w = model.addMVar((m,n,d), vtype=GRB.BINARY, name = 'w') # variable for linearization

# set objective function
model.setObjective((sum((b[j]*T[i][j] - c[j]) * x[i,j] for j in range(n) for i in range(m))), GRB.MAXIMIZE)

# add constraints
model.addConstrs((((sum(c[j]*x[i,j] for i in range(m))) <= C[j]) for j in range(n)))

model.addConstrs((((sum(z[i,j,l] for j in range(n))) <= 1) for i in range(m) for l in range(d)))

model.addConstrs((((l+p[j])*w[i,j,l] <= d) for i in range(m) for j in range(n) for l in s[j]))

for j in range(n):
    possible_running_times = set()
    for start_time in s[j]:
        for day in range(start_time,min(start_time+p[j],d)):
            possible_running_times.add(day)
    model.addConstrs(((z[i,j,l] == 0) for i in range(m) for l in range(d) if l not in possible_running_times))

model.addConstrs((((sum(z[i,j,l] for l in range(start_time,min(start_time+p[j],d)))) >= p[j]*w[i,j,start_time]) for j in range(n) for i in range(m) for start_time in s[j]))

model.addConstrs((((sum(v[j,l] for l in s[j])) == 1) for j in range(n)))

model.addConstrs(((v[j,l] == 0) for j in range(n) for l in range(d) if l not in s[j]))

model.addConstrs(((w[i,j,l] >= x[i,j]+v[j,l]-1) for i in range(m) for j in range(n) for l in range(d)))

model.addConstrs(((w[i,j,l] <= x[i,j]) for i in range(m) for j in range(n) for l in range(d)))

model.addConstrs(((w[i,j,l] <= v[j,l]) for i in range(m) for j in range(n) for l in range(d)))

# solve model
model.optimize()

# get attributes
if model.SolCount > 0:
    x_values = []
    v_values = []
    for k in range(m*n):
        var = model.getVarByName("x[{}]".format(k))
        x_values.append((var.varName, var.x))
    for k in range(n*D):
        var = model.getVarByName("v[{}]".format(k))
        if var.x == 1.0:
            v_values.append(var.varName)

cwd = os.getcwd()
path = '{}/output'.format(cwd)
if not os.path.exists(path):
    os.makedirs(path)
    
# write schedule into file       
with open('{}/msptumkp.csv'.format(path), mode = 'w') as schedule:
    writer = csv.writer(schedule, delimiter = ';')
    writer.writerow(['campaign','customer','starting time', 'processing time'])
    
    for value in x_values:
        if value[1] == 1.0:
            count = int(value[0][value[0].find('[')+1:value[0].find(']')])
            customer = math.floor(count/n)
            campaign = count%n
            indice = int(v_values[campaign][v_values[campaign].find('[')+1:v_values[campaign].find(']')])
            starting = indice%d
            writer.writerow(campaign, customer, starting, p[campaign])
    
schedule.close()

# export model
# model.write('msptumkp.lp')

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (linux64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 89029 rows, 45725 columns and 205825 nonzeros
Model fingerprint: 0xf0a1d9ec
Variable types: 0 continuous, 45725 integer (45725 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [2e-01, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective -0.0000000
Presolve removed 89029 rows and 45725 columns
Presolve time: 0.04s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.05 seconds
Thread count was 1 (of 12 available processors)

Solution count 1: -0 
No other solutions better than -0

Optimal solution found (tolerance 1.00e-04)
Best objective -0.000000000000e+00, best bound -0.000000000000e+00, gap 0.0000%
CPU times: user 4min 22s, sys: 5.6 s, total: 4min 28s
Wall time: 4min 27s
